In [2]:
"""Specification Test:  Non-nested hypotheses: Model 1.

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("../Data/swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)

B_SM_LOW_INCOME = Beta('B_SM_LOW_INCOME',0,-10,10,0)
B_SM_HIGH_INCOME	 = Beta('B_SM_HIGH_INCOME',0,-10,10,0)
B_CAR_LOW_INCOME = Beta('B_CAR_LOW_INCOME',0,-10,10,0)
B_CAR_HIGH_INCOME	 = Beta('B_CAR_HIGH_INCOME',0,-10,10,0)
B_TRAIN_LOW_INCOME = Beta('B_TRAIN_LOW_INCOME',0,-10,10,0)
B_TRAIN_HIGH_INCOME	 = Beta('B_TRAIN_HIGH_INCOME',0,-10,10,0)


# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
LOW_INCOME  = DefineVariable('LOW_INCOME', INCOME  ==  1,database )
HIGH_INCOME  = DefineVariable('HIGH_INCOME', INCOME  ==  3,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR \
        + B_CAR_LOW_INCOME * LOW_INCOME + B_CAR_HIGH_INCOME * HIGH_INCOME
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST \
        + B_TRAIN_LOW_INCOME * LOW_INCOME + B_TRAIN_HIGH_INCOME * HIGH_INCOME
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST \
        + B_SM_LOW_INCOME * LOW_INCOME + B_SM_HIGH_INCOME * HIGH_INCOME

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Estimation of the model
logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "SpecTest_SM_M1"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.194
ASC_SM=	0.213
B_CAR_COST=	-0.00743
B_CAR_HIGH_INCOME=	0.242
B_CAR_LOW_INCOME=	0.0566
B_CAR_TIME=	-0.013
B_FIRST=	0.254
B_GA=	0.581
B_HE=	-0.00633
B_SENIOR=	-1.76
B_SM_COST=	-0.0118
B_SM_HIGH_INCOME=	0.248
B_SM_LOW_INCOME=	-0.307
B_SM_TIME=	-0.0108
B_TRAIN_COST=	-0.027
B_TRAIN_HIGH_INCOME=	-0.49
B_TRAIN_LOW_INCOME=	0.251
B_TRAIN_TIME=	-0.00819
                        Value   Std err     t-test       p-value  \
ASC_CAR             -0.194316  0.157646  -1.232607  2.177223e-01   
ASC_SM               0.212811  0.127732   1.666074  9.569859e-02   
B_CAR_COST          -0.007427  0.001073  -6.921070  4.482414e-12   
B_CAR_HIGH_INCOME    0.242333  0.054249   4.467073  7.929695e-06   
B_CAR_LOW_INCOME     0.056563  0.092040   0.614553  5.388501e-01   
B_CAR_TIME          -0.012952  0.000811 -15.970541  0.000000e+00   
B_FIRST              0.253991  0.077980   3.257116  1.125503e-03   
B_GA                 0.581077  0.188935   3.075544  2.101192e-03   
B_HE                -0.0063